In [1]:
from pydantic import BaseModel,Field
from langchain.agents import create_agent
from langchain_ollama import ChatOllama
model=ChatOllama(model="gpt-oss:120b-cloud")
class ContactInfo(BaseModel):
    """Contact information for a person."""
    name: str = Field(description="The name of the person")
    email: str = Field(description="The email address of the person")
    phone: str = Field(description="The phone number of the person")

agent = create_agent(
    model=model,
    tools=[],
    response_format=ContactInfo  # Auto-selects ProviderStrategy
)

result = agent.invoke({
    "messages": [{"role": "user", "content": "Extract contact info from: John Doe, john@example.com, (555) 123-4567"}]
})

result["structured_response"]
# ContactInfo(name='John Doe', email='john@example.com', phone='(555) 123-4567')

ContactInfo(name='John Doe', email='john@example.com', phone='(555) 123-4567')

In [9]:
from langchain_ollama import ChatOllama
model = ChatOllama(model="gpt-oss:120b-cloud")
chain=model.with_structured_output(ContactInfo)
response=chain.invoke("Extract contact info from: John Doe, john@example.com, (555) 123-4567")
response

ContactInfo(name='John Doe', email='john@example.com', phone='(555) 123-4567')

Tool calling strategy


In [5]:
from pydantic import BaseModel, Field
from typing import Literal
from langchain.agents import create_agent
from langchain.agents.structured_output import ToolStrategy
from langchain_ollama import ChatOllama
model = ChatOllama(model="gpt-oss:120b-cloud")
class ProductReview(BaseModel):
    """Analysis of a product review."""
    rating: int | None = Field(description="The rating of the product", ge=1, le=5)
    sentiment: Literal["positive", "negative"] = Field(description="The sentiment of the review")
    key_points: list[str] = Field(description="The key points of the review. Lowercase, 1-3 words each.")

agent = create_agent(
    model=model,
    tools=[],
    response_format=ToolStrategy(ProductReview)
)

result = agent.invoke({
    "messages": [{"role": "user", "content": "Analyze this review: 'Great product: 5 out of 5 stars. Fast shipping, but expensive'"}]
})
result["structured_response"].rating
# ProductReview(rating=5, sentiment='positive', key_points=['fast shipping', 'expensive'])

5

Multiple structured outputs error


In [10]:
from pydantic import BaseModel, Field
from typing import Union
from langchain.agents import create_agent
from langchain.agents.structured_output import ToolStrategy
from langchain_ollama import ChatOllama
model = ChatOllama(model="gpt-oss:120b-cloud")
class ContactInfo(BaseModel):
    name: str = Field(description="Person's name")
    email: str = Field(description="Email address")

class EventDetails(BaseModel):
    event_name: str = Field(description="Name of the event")
    date: str = Field(description="Event date")

agent = create_agent(
    model=model,
    tools=[],
    response_format=ToolStrategy(Union[ContactInfo, EventDetails])  # Default: handle_errors=True
)

response=agent.invoke({
    "messages": [{"role": "user", "content": "Extract info: John Doe (john@email.com) is organizing Tech Conference on March 15th"}]
})
response

{'messages': [HumanMessage(content='Extract info: John Doe (john@email.com) is organizing Tech Conference on March 15th', additional_kwargs={}, response_metadata={}, id='bb2e0ff1-6622-42e6-89dd-4647b2a53e6f'),
  AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'gpt-oss:120b-cloud', 'created_at': '2025-10-16T09:10:51.491130819Z', 'done': True, 'done_reason': 'stop', 'total_duration': 1281332102, 'load_duration': None, 'prompt_eval_count': 174, 'prompt_eval_duration': None, 'eval_count': 144, 'eval_duration': None, 'model_name': 'gpt-oss:120b-cloud', 'model_provider': 'ollama'}, id='lc_run--2a381739-2cdf-4540-958b-7a89ce65760f-0', tool_calls=[{'name': 'ContactInfo', 'args': {'email': 'john@email.com', 'name': 'John Doe'}, 'id': '3ff8d94c-57a8-4cc7-8ccd-f836b0e16dcb', 'type': 'tool_call'}], usage_metadata={'input_tokens': 174, 'output_tokens': 144, 'total_tokens': 318}),
  ToolMessage(content="Returning structured response: name='John Doe' email='john@email.com'", n

In [ ]:
from pydantic import BaseModel, Field
from langchain.agents import create_agent
from langchain.agents.structured_output import ToolStrategy
from langchain_ollama import ChatOllama
#model = ChatOllama(model="gpt-oss:120b-cloud")
model = ChatOllama(model="llama3.2:latest")
class ProductRating(BaseModel):
    rating: int | None = Field(description="Rating from 1-5", ge=1, le=5)
    comment: str = Field(description="Review comment")

agent = create_agent(
    model=model,
    tools=[],
    response_format=ToolStrategy(
        ProductRating,
        handle_errors="Please provide a valid rating between 1-5 and include a comment."
        ),  # Default: handle_errors=True
    system_prompt="You are a helpful assistant that parses product reviews. Do not make any field or value up."
)

agent.invoke({
    "messages": [{"role": "user", "content": "Parse this: Amazing product, 10/10!"}]
})

ResponseError: registry.ollama.ai/library/deepseek-r1:7b does not support tools (status code: 400)